Run face and object detection
```
docker run -e VISION-DETECTION=True -e VISION-FACE=True -e MODE=High -d \
      -v localstorage:/datastore -p 5000:5000 \
       --name deepstack deepquestai/deepstack:noavx
```

In [ ]:
import requests
from PIL import Image, ImageDraw

In [ ]:
def draw_box(draw, prediction, text='', color=(255, 0, 0)):
    """Draw bounding box on image."""
    (left, right, top, bottom) = (prediction['x_min'], prediction['x_max'], prediction['y_min'], prediction['y_max'])
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)], width=5, fill=color)
    if text:
        draw.text((left, abs(top-15)), text, fill=color)

In [ ]:
def format_confidence(confidence):
    """Takes a confidence from the API like 
       0.55623 and returne 55.6 (%).
    """
    return round(float(confidence)*100, 1)

format_confidence(0.55623)

In [ ]:
# IP_ADDR = 'localhost'
IP_ADDR = '192.168.1.252'
PORT = 80

## Face detection
https://deepstackpython.readthedocs.io/en/latest/facedetection.html

Register a face - this can take a long time

In [ ]:
image_data = open("idris.jpg","rb").read()
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/face/register",files={"image":image_data}, data={"userid":"Idris Elba"})

In [ ]:
response.json()

Run a face detection - fast option

In [ ]:
image_data = open("couple.jpg","rb").read()
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/face",files={"image":image_data})

In [ ]:
response.json()

In [ ]:
predictions = response.json()['predictions']

In [ ]:
image = Image.open("couple.jpg")
draw = ImageDraw.Draw(image)

for prediction in predictions:
    prediction_confidence = format_confidence(prediction['confidence'])
    draw_box(draw, prediction, str(prediction_confidence))

display(image)

Run face recognition - this can also take a long time

In [ ]:
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/face/recognize", files={"image":image_data})

In [ ]:
response.json()

## Object detection
https://deepstackpython.readthedocs.io/en/latest/objectdetection.html

In [ ]:
ls

In [ ]:
%%time
img_file_test = "test-image3.jpg"
print(img_file_test)
image_data = open(img_file_test,"rb").read()
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/detection",files={"image":image_data})

In [ ]:
predictions = response.json()['predictions']
predictions

In [ ]:
image = Image.open(img_file_test)
draw = ImageDraw.Draw(image)

for prediction in predictions:
    prediction_confidence = format_confidence(prediction['confidence'])
    draw_box(draw, prediction, str(prediction_confidence))

display(image)

## Add auth
API key can be added to deepstack by including `-e API-KEY=Mysecretkey`. It is necessary to unset the admin key by passing it as an empty string `-e API-KEY=""`

The API Key protects all recognition and detection endpoints including face, scene, object detection and custom models. 

In [ ]:
image_data = open("couple.jpg","rb").read()
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/face",files={"image":image_data}, data={"api_key":""})

If no key provided

In [ ]:
response.status_code

In [ ]:
response.json()

Provide the key in `data`

In [ ]:
response = requests.post(f"http://{IP_ADDR}:{PORT}/v1/vision/face",
                         files={"image":image_data}, 
                         data={"api_key":"Mysecretkey"})

In [ ]:
response.json()

The default api_key is a blank string and this can be passed with any reqyest where an api_key has not been set